In [1]:
"""
Block-partitioned LU method: 
            A=LU where 
                    L is lower triangular with the Id in the diagonal
                    U is upper triangular 

No permutation matrix (i.e. Gauss without pivoting!) -> It may not exists for concrete matrices
"""

import numpy as np
import scipy.linalg

def lunopiv(A,n,ptol):
    for k in range(0,n-1):
        pivot = A[k,k]
        if abs(pivot) < ptol:
           print('zero pivot encountered')
           break
        for i in range(k+1,n):
            A[i,k] = A[i,k]/pivot
            for j in range(k+1,n):
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
    L = np.eye(n)+np.tril(A,-1)
    U = np.triu(A)
    return L,U


In [2]:
"""
MAIN
"""

n=6;
A=np.zeros((n,n))
for i in range(0,n):
    for j in range (0,n):
        A[i,j]=np.random.normal(0,10)

LU=np.zeros((n,n)) #it will contain the final LU block factorization


r=3; #size of A11 block
nblocks = int(n/r) #number of blocks (assuming it is divisible!)
Aij=np.zeros((nblocks,nblocks),np.matrix) #blocks -> for each pair (i,j) it contains a matrix 

S=np.copy(A)  #it will contain the Schur complement, to perform LU factorization in the next step

for i in range(0,nblocks-1):   
    Aij[0,0] = np.copy(S[0:r,0:r]);   Aij[0,1] = np.copy(S[0:r,r:n]);
    Aij[1,0] = np.copy(S[r:n,0:r]);   Aij[1,1] = np.copy(S[r:n,r:n]);
    L11,U11 = lunopiv(Aij[0,0],r,1.e-16) #note that Aij[0,0] will be overwritten in lunopiv

    print('%d %24.16e\n' % ( i, np.linalg.norm(np.dot(L11,U11)-S[0:r,0:r],np.inf) ))
    U12=np.linalg.solve(L11,Aij[0,1])      #Forward substitution! n-linear systems
    L21t=np.linalg.solve(U11.T,Aij[1,0].T) #Forward substitution! n-linear systems

    LU[i*r:(i+1)*r,i*r:(i+1)*r]=np.copy(np.tril(L11,-1)+U11);     #Copy in LU
    LU[i*r:(i+1)*r,(i+1)*r:n]=np.copy(U12);
    LU[(i+1)*r:n,i*r:(i+1)*r]=np.copy(L21t.T);

    S=Aij[1,1]-np.dot(L21t.T,U12)  #Schur complement of Aij[0,0]
    

Aij[0,0] = S[0:r,0:r]; 
L11,U11 = lunopiv(Aij[0,0],r,1.e-16) #Aij[0,0] overwritten in lunopiv
LU[n-r:n,n-r:n]=np.copy(np.tril(L11,-1)+U11);     #Copy in LU

print('LU factorization ( r=%d) \n' % (r))
for i in range (0,n):
    for j in range (0,n):
        print(" %15.8e " % LU[i,j], end="") 
    print('\n')
print('CHECK: %24.16e \n' % (np.linalg.norm(A-np.dot(np.eye(n)+np.tril(LU,-1),np.triu(LU)),np.inf)))

0   5.5511151231257827e-16

LU factorization ( r=3) 

  9.96077393e+00   7.42553122e-01   7.89280382e+00   8.12390111e+00  -9.69704994e+00   9.00251834e-01 

  2.95640476e-01  -6.20773122e+00  -1.51856878e+00   3.01601680e-01   1.83459820e+01  -1.35148741e+01 

 -1.42837561e+00  -9.17409111e-01   3.57100282e+00   2.01691864e+01   5.64803218e+00  -8.08728216e+00 

 -4.40848656e-01   1.61495454e+00   2.09854053e+00  -3.32461809e+01  -3.94736481e+01   4.56795950e+01 

 -2.43024173e+00  -4.13091954e-01   5.88938639e+00   3.07845204e+00   7.62826868e+01  -9.62500607e+01 

 -7.40005693e-01   5.50200601e-01  -4.05493243e-01  -1.27398051e-01  -3.05071766e-01  -2.85286808e+01 

CHECK:   3.1030733538273125e-14 



In [3]:
import numpy as np

"""
Standard matrix multiplication

1) Involves 2n^3 flop (counting additions and multiplications separately)
2) Produces and consumes only  3n^2 data values

Problem: data is not reused! for each i (outer loop) the data of A[i,] is not reused any more! 

Idea: Take advantage of this and access small blocks (that can be stored or even move to fast memory acces area).
"""

################################


def blockmatprod(A,B):
    C=np.zeros((n,n)); 
    for i in range (0,N): 
        for j in range (0,N): # <- here we load block C[i,j] into fast  memory
            for k in range (0,N): # <- here we load block A[i,k], B[k,j] into fast memory
                for ii in range (i*b,(i+1)*b):
                    for jj in range (j*b,(j+1)*b):  
                        for kk in range (k*b,(k+1)*b):
                            C[ii,jj]=C[ii,jj]+A[ii,kk]*B[kk,jj];
            # <- here we write block C[ii,jj] into slow memory
    return C;


In [4]:
"""
MAIN

Sup n=Nb, with b=block size small enough so that we can fit one block of A,B and C in Cache Memory (Fast Memory area)
"""
p=7
n=2**p  #dimension (e.g. a power of 2)

A=np.random.rand(n,n); B=np.random.rand(n,n)  #random matrices

C1=np.dot(A,B);  print('Direct computation %d\n' % n);  #direct multiplication of matrices


for pp in range (0,p+1):
     N=2**pp; b=int(n/N); 
     C2=blockmatprod(A,B);  #block matrix multiplication
     print(' %4d %4d %4d %24.16e\n'%(n,b,N,np.linalg.norm(C2-C1,np.inf)))

Direct computation 128

  128  128    1   3.0553337637684308e-13

  128   64    2   3.0553337637684308e-13

  128   32    4   3.0553337637684308e-13

  128   16    8   3.0553337637684308e-13

  128    8   16   3.0553337637684308e-13

  128    4   32   3.0553337637684308e-13

  128    2   64   3.0553337637684308e-13

  128    1  128   3.0553337637684308e-13



In [5]:
import numpy as np
import time

"""
Standard matrix multiplication

1) Involves 2n^3 flop (counting additions and multiplications separately)
2) Produces and consumes only  3n^2 data values

Problem: data is not reused! for each i (outer loop) the data of A[i,] is not reused any more! 

Idea: Take advantage of this and access small blocks (that can be stored or even move to fast memory acces area).
"""

################################


def blockmatprod(A,B):
    C=np.zeros((n,n)); 
    for i in range (0,N): 
        for j in range (0,N): # <- here we load block C[i,j] into fast  memory
            for k in range (0,N): # <- here we load block A[i,k], B[k,j] into fast memory
                for ii in range (i*b,(i+1)*b):
                    for jj in range (j*b,(j+1)*b):  
                        for kk in range (k*b,(k+1)*b):
                            C[ii,jj]=C[ii,jj]+A[ii,kk]*B[kk,jj];
            # <- here we write block C[ii,jj] into slow memory
    return C;


In [6]:
"""
MAIN

Sup n=Nb, with b=block size small enough so that we can fit one block of A,B and C in Cache Memory (Fast Memory area)
"""
p=7
n=2**p  #dimension (e.g. a power of 2)

A=np.random.rand(n,n); B=np.random.rand(n,n)  #random matrices

st=time.perf_counter();  C1=np.dot(A,B);  et=time.perf_counter(); print('Direct computation %d %e\n' % (n,et-st));  #direct multiplication of matrices


for pp in range (0,p+1):
    st=time.perf_counter(); N=2**pp; b=int(n/N); 
    C2=blockmatprod(A,B); et=time.perf_counter();  #block matrix multiplication
    print(' %4d %4d %4d %24.16e %24.16e\n'%(n,b,N,np.linalg.norm(C2-C1,np.inf),et-st))



Direct computation 128 2.401840e-03

  128  128    1   2.9132252166164108e-13   1.6053778790000024e+00

  128   64    2   2.9132252166164108e-13   1.5020232950000150e+00

  128   32    4   2.9132252166164108e-13   1.5319009319999850e+00

  128   16    8   2.9132252166164108e-13   1.5493773630000192e+00

  128    8   16   2.9132252166164108e-13   1.5936431070000197e+00

  128    4   32   2.9132252166164108e-13   1.7365111550000165e+00

  128    2   64   2.9132252166164108e-13   2.2001794939999968e+00

  128    1  128   2.9132252166164108e-13   4.2317007159999775e+00



In [7]:
import numpy as np


"""
ijk -> the inner loop is an inner product of two vectors (a dot product), 
                 requires acces to a row of A and a column of B

       the middle loop performs a (vector x matrix) operation 
                 requires acces to a row of A and the matrix B
"""

def ijk(A,B,C):
   for i in range (0,n):
       for j in range (0,n):
           for k in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C


def ijk_improved(A,B,C):
   for i in range (0,n):
       for j in range (0,n):
           C[i,j] = np.dot(A[i,:],B[:,j]) + C[i,j]
   return C


"""
jik -> similar to ijk, the inner loop is also an inner/dot product
                 requires acces to a row of A and a column of B

       the middle loop performs a (matrix x vector) operation
                 requires acces to the matrix A and a column of B
"""
def jik(A,B,C):
   for j in range (0,n):
       for i in range (0,n):
           for k in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C


"""
ikj -> the inner loop is the multiplication of a number by a row of B (saxpy)
       the middle loop is row gaxpy operation 
"""

def ikj(A,B,C):
   for i in range (0,n):
       for k in range (0,n):
           for j in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C


def ikj_improved(A,B,C):
   for i in range (0,n):
       for k in range (0,n):
           aux = A[i,k]               # only one memory access!
           for j in range (0,n):
               C[i,j] = aux*B[k,j] + C[i,j]
   return C


"""
jki -> inner loop (saxpy)
       middle loop (column gaxpy)
       similar to ikj
"""

def jki(A,B,C):
   for j in range (0,n):
       for k in range (0,n):
           for i in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C
        

"""
kij -> inner loop (saxpy)
       middle loop (row outer product or tensorial product) 

       Rec: Given two vectors, a = [a0, a1, ..., aM] and b = [b0, b1, ..., bN], the outer product is the matrix

            [[a0*b0  a0*b1 ... a0*bN ]
             [a1*b0    .
             [ ...          .
             [aM*b0            aM*bN ]]


"""
def kij(A,B,C):
   for k in range (0,n):
       for i in range (0,n):
           for j in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C
 
def kij_improved(A,B,C):
   for k in range (0,n):
       C = np.outer(A[:,k],B[k,:]) + C
   return C
 

"""
kji -> similar to kij
       middle loop (column outer product)
"""    
def kji(A,B,C):
   for k in range (0,n):
       for j in range (0,n):
           for i in range (0,n):
               C[i,j] = A[i,k]*B[k,j] + C[i,j]
   return C
 


"""
MAIN
"""

n=5

A=np.random.rand(n,n)
B=np.random.rand(n,n)
C=np.zeros((n,n))

print(A); print("\n")
print(B); print("\n")
print(C); print("\n")

C=kij_improved(A,B,C)
print(C); print("\n")
print(np.dot(A,B))

[[0.98872887 0.93599929 0.28295141 0.15227144 0.22017396]
 [0.57614482 0.47009269 0.94376049 0.34273278 0.10452814]
 [0.55997936 0.97077753 0.70271993 0.10429478 0.50498742]
 [0.45907363 0.75436761 0.82869488 0.49534289 0.13028859]
 [0.70151658 0.9662546  0.88483214 0.71470992 0.86164712]]


[[0.19691348 0.59267875 0.07824446 0.54079708 0.42140135]
 [0.95841775 0.47461323 0.27217709 0.81120493 0.84547831]
 [0.80959467 0.82447185 0.19807464 0.80363688 0.31104594]
 [0.2667039  0.35727293 0.109384   0.42473703 0.51004715]
 [0.88646416 0.41736233 0.68176326 0.3099388  0.51213919]]


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


[[1.55663605 1.40981649 0.55492819 1.65429488 1.48645499]
 [1.51212794 1.50876021 0.46871655 1.6293283  1.16213708]
 [2.08506529 1.62002896 0.80291973 1.85587942 1.58714468]
 [1.7319101  1.54470178 0.54839355 1.77695412 1.40839199]
 [2.73500423 2.21885594 1.15876214 2.44491399 2.19360979]]


[[1.55663605 1.40981649 0.55